In [2]:
from mobo.surrogate_model import BoTorchSurrogateModelReapeat, BoTorchSurrogateModel
from mobo.transformation import StandardTransform
import numpy as np

In [5]:
import torch
import pandas as pd
from pathlib import Path

sm = BoTorchSurrogateModelReapeat(n_var=1, n_obj=1, alpha=0.9, n_w=11)

# .\result\exp4d\optimization_4\marsdet\0\state_dict_noise.pt

path = Path("./result/exp4d/optimization_4/marsdet/0/")

state_dict = torch.load(path / "state_dict.pt")
state_dict_noise = torch.load(path / "state_dict_noise.pt")

# [
#     "iterID",
#     "x1",
#     "x2",
#     "x3",
#     "x4",
#     "f1",
#     "rho_f1",
#     "mvar_f1",
#     "f2",
#     "rho_f2",
#     "mvar_f2",
#     "f3",
#     "rho_f3",
#     "mvar_f3",
#     "Expected_f1",
#     "Uncertainty_f1",
#     "Acquisition_f1",
#     "Expected_f2",
#     "Uncertainty_f2",
#     "Acquisition_f2",
#     "Expected_f3",
#     "Uncertainty_f3",
#     "Acquisition_f3",
#     "Hypervolume_indicator",
#     "MVaR_Hypervolume_indicator",
# ]
eval_samples = pd.read_csv(path / "EvaluatedSamples.csv")
eval_samples.columns.to_list()
train_X = eval_samples[["x1", "x2", "x3", "x4"]].values
train_Y = eval_samples[["f1", "f2", "f3"]].values
train_Yvar = eval_samples[["rho_f1", "rho_f2", "rho_f3"]].values

import torch
#np to torch
train_X = torch.tensor(train_X, dtype=torch.float)
train_Y = torch.tensor(train_Y, dtype=torch.float)

# botorch cross validation
from botorch import cross_validation
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.cross_validation import batch_cross_validation, gen_loo_cv_folds
cv_folds = gen_loo_cv_folds(train_X, train_Y)
cv_results = batch_cross_validation(
    SingleTaskGP,
    ExactMarginalLogLikelihood,
    cv_folds,
)
cv_results

CVResults(model=SingleTaskGP(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): MaternKernel(
      (lengthscale_prior): GammaPrior()
      (raw_lengthscale_constraint): Positive()
    )
    (outputscale_prior): GammaPrior()
    (raw_outputscale_constraint): Positive()
  )
), posterior=<botorch.posteriors.gpytorch.GPyTorchPosterior object at 0x000001FCACBF82B0>, observed_Y=tensor([[[ -0.2301,  -3.4733,  -0.9930]],

        [[ -0.9098,  -2.2933,  -6.0390]],

        [[ -0.0455,  -0.7600,  -2.3805]],

        [[ -0.9325,  -2.3733,  -4.0239]],

        [[ -0.0301,  -1.1633,  -2.0240]],

        [[ -0.9388,  -2.2800,  -4.7802]],

        [[ -0.0433,  -1.1833,  -3.8102]],

        [[ -0.1373,  -3.9300,  -0.5810]],

        [[ -0.0827,  -4.1900,  -0.4130]],

        [[ -0.9369,  -2.3600,  -4